# 基于机器学习数据库飞速上线AI应用——RUL

剩余使用寿命（remaining useful life，RUL），指一个系统正常工作一段时间后,能够正常运转的时间。借助RUL,工程师可以安排维护时间、优化运行效率并避免计划外停机。因此,预测RUL是预测性维护计划中的首要任务。 
本次的任务就是开发一个通过机器学习模型进行剩余使用寿命预测的实时智能应用。我们使用NASA提供的[Turbofan Engine Degradation Simulation Data Set](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan)，作为训练集与测试集。

整个应用开发是基于[notebook](http://ipython.org/notebook.html)。


## 初始化环境
整个初始化过程包含安装fedb，以及相关运行环境，初始化脚步可以参考https://github.com/4paradigm/DemoApps/blob/main/predict-remaining-useful-life-nb/demo/init.sh

In [1]:
!cd demo && sh init.sh

ZooKeeper JMX enabled by default
Using config: /home/jovyan/work/zookeeper-3.4.14/bin/../conf/zoo.cfg
Starting zookeeper ... already running as process 729.
Starting tablet ... tablet already running as process 798.
Starting nameserver ... nameserver already running as process 851.
2021-06-17 02:35:07,473:2351(0x7f79af3a0a00):ZOO_INFO@log_env@753: Client environment:zookeeper.version=zookeeper C client 3.4.14
2021-06-17 02:35:07,474:2351(0x7f79af3a0a00):ZOO_INFO@log_env@757: Client environment:host.name=m7-pce-dev01
2021-06-17 02:35:07,474:2351(0x7f79af3a0a00):ZOO_INFO@log_env@764: Client environment:os.name=Linux
2021-06-17 02:35:07,474:2351(0x7f79af3a0a00):ZOO_INFO@log_env@765: Client environment:os.arch=3.10.0-1127.18.2.el7.x86_64
2021-06-17 02:35:07,474:2351(0x7f79af3a0a00):ZOO_INFO@log_env@766: Client environment:os.version=#1 SMP Sun Jul 26 15:27:06 UTC 2020
2021-06-17 02:35:07,474:2351(0x7f79af3a0a00):ZOO_INFO@log_env@774: Client environment:user.name=(null)
2021-06-17 02:35:07,

## 导入行程历史数据到fedb

使用fedb进行时序特征计算是需要历史数据的，所以我们将历史数据导入到fedb，以便实时推理可以使用历史数据进行特征推理，导入代码可以参考https://github.com/4paradigm/DemoApps/blob/main/predict-taxi-trip-duration-nb/demo/import.py
这里使用data/test_FD004.txt作为历史数据。

In [2]:
!cd demo && python3 import.py
# 只导入某一个engine的？

2021-06-17 02:35:08,766:2366(0x7f4dc08b2740):ZOO_INFO@log_env@753: Client environment:zookeeper.version=zookeeper C client 3.4.14
2021-06-17 02:35:08,766:2366(0x7f4dc08b2740):ZOO_INFO@log_env@757: Client environment:host.name=m7-pce-dev01
2021-06-17 02:35:08,766:2366(0x7f4dc08b2740):ZOO_INFO@log_env@764: Client environment:os.name=Linux
2021-06-17 02:35:08,766:2366(0x7f4dc08b2740):ZOO_INFO@log_env@765: Client environment:os.arch=3.10.0-1127.18.2.el7.x86_64
2021-06-17 02:35:08,766:2366(0x7f4dc08b2740):ZOO_INFO@log_env@766: Client environment:os.version=#1 SMP Sun Jul 26 15:27:06 UTC 2020
2021-06-17 02:35:08,766:2366(0x7f4dc08b2740):ZOO_INFO@log_env@774: Client environment:user.name=(null)
2021-06-17 02:35:08,766:2366(0x7f4dc08b2740):ZOO_INFO@log_env@782: Client environment:user.home=/root
2021-06-17 02:35:08,766:2366(0x7f4dc08b2740):ZOO_INFO@log_env@794: Client environment:user.dir=/home/jovyan/work/rul/demo
2021-06-17 02:35:08,766:2366(0x7f4dc08b2740):ZOO_INFO@zookeeper_init@827: Initi

## 生成训练特征矩阵
模型训练需要训练数据，以下是生成使用的的代码

* 训练特征矩阵生成脚本代码 https://github.com/4paradigm/DemoApps/blob/main/predict-remaining-useful-life-nb/demo/gene_train_by_ft.py 
* 训练数据 train_FD004.txt

整个任务最终会生成训练用的特征矩阵，保存为simple_fm.csv。

In [3]:
# !cd demo && python3 gene_train_by_ft.py

## 使用训练的模型搭建链接fedb的实时推理http服务——TODO

基于上一步生成的模型和fedb历史数据，搭建一个实时推理服务，整个推理服务代码参考https://github.com/4paradigm/DemoApps/blob/main/predict-taxi-trip-duration-nb/demo/predict_server.py

hw: 先跑通逻辑，再做成服务。

In [4]:
# !cd demo && sh start_predict_server.sh # TODO
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd
import utils

fm = pd.read_csv('demo/simple_fm.csv', index_col='engine_no')
X = fm.copy().fillna(0)
y = X.pop('remaining_useful_life')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)

# skip baselines

reg = RandomForestRegressor(n_estimators=100)
reg.fit(X_train, y_train)

preds = reg.predict(X_test)
scores = mean_absolute_error(preds, y_test)
print('Mean Abs Error: {:.2f}'.format(scores))

high_imp_feats = utils.feature_importances(X, reg, feats=10)

fm.head()

Mean Abs Error: 46.95
1: MAX(recordings.sensor_measurement_4) [0.171]
2: MAX(recordings.sensor_measurement_15) [0.113]
3: MAX(recordings.sensor_measurement_11) [0.091]
4: MAX(recordings.sensor_measurement_13) [0.078]
5: MAX(recordings.operational_setting_2) [0.078]
6: MAX(recordings.sensor_measurement_3) [0.069]
7: MAX(recordings.sensor_measurement_21) [0.060]
8: MAX(recordings.sensor_measurement_9) [0.051]
9: MAX(recordings.sensor_measurement_2) [0.047]
10: MAX(recordings.sensor_measurement_8) [0.035]
-----



,MAX(recordings.operational_setting_1),MAX(recordings.operational_setting_2),MAX(recordings.operational_setting_3),MAX(recordings.sensor_measurement_1),MAX(recordings.sensor_measurement_10),MAX(recordings.sensor_measurement_11),MAX(recordings.sensor_measurement_12),MAX(recordings.sensor_measurement_13),MAX(recordings.sensor_measurement_14),MAX(recordings.sensor_measurement_15),...,MAX(recordings.cycles.MAX(recordings.sensor_measurement_20)),MAX(recordings.cycles.MAX(recordings.sensor_measurement_21)),MAX(recordings.cycles.MAX(recordings.sensor_measurement_3)),MAX(recordings.cycles.MAX(recordings.sensor_measurement_4)),MAX(recordings.cycles.MAX(recordings.sensor_measurement_5)),MAX(recordings.cycles.MAX(recordings.sensor_measurement_6)),MAX(recordings.cycles.MAX(recordings.sensor_measurement_7)),MAX(recordings.cycles.MAX(recordings.sensor_measurement_8)),MAX(recordings.cycles.MAX(recordings.sensor_measurement_9)),remaining_useful_life
engine_no,,,,,,,,,,,,,,,,,,,,,
1,42.0067,0.8420,100.0,518.67,1.3,47.43,521.18,2388.06,8136.59,10.9141,...,39.14,23.4578,1585.87,1404.99,14.62,21.59,553.15,2387.96,9053.86,220
2,42.0079,0.8420,100.0,518.67,1.3,47.47,522.54,2388.14,8138.46,10.9057,...,39.15,23.4692,1591.12,1406.87,14.62,21.61,555.00,2388.07,9056.19,198
3,42.0071,0.8418,100.0,518.67,1.3,47.36,524.69,2388.14,8147.30,10.8552,...,39.36,23.6131,1591.12,1409.55,14.62,21.61,556.80,2388.14,9067.99,206
4,42.0080,0.8419,100.0,518.67,1.3,47.54,522.30,2388.19,8146.95,10.9442,...,39.36,23.6131,1596.26,1411.32,14.62,21.61,556.80,2388.14,9070.01,173
5,42.0065,0.8417,100.0,518.67,1.3,47.72,521.43,2388.24,8131.69,10.9454,...,39.36,23.6131,1598.72,1417.79,14.62,21.61,556.80,2388.21,9070.01,92


## 通过http请求发送一个推理请求——TODO
hw: 先直连fedb

In [39]:
# fm2
fm2 = pd.read_csv(
    'demo/test_fm.csv',
    index_col=False,)

data2 = utils.load_data('demo/data/test_FD004.txt')
d2 = data2.drop(['index'],axis=1)
from pandasql import sqldf
def pysqldf(q):
    "add this to your script if you get tired of calling locals()"
    return sqldf(q, globals())

fm2_sql = pysqldf("""
select 
engine_no,
MAX(operational_setting_1),
MAX(rcm_operational_setting_1) 
from d2 as recordings 
join (select time_in_cycles, MAX(operational_setting_1) as rcm_operational_setting_1 from d2 group by time_in_cycles) as cycles
on recordings.time_in_cycles=cycles.time_in_cycles
group by engine_no
;
""")
# print(list(fm2_sql))
# print(list(fm2))

print((fm2_sql['MAX(operational_setting_1)']==fm2['MAX(recordings.operational_setting_1)']).value_counts())
print((fm2_sql['MAX(rcm_operational_setting_1)']==fm2['MAX(recordings.cycles.MAX(recordings.operational_setting_1))']).value_counts())

# /*用另一张表？单独追每一个time_in_cycles，各种max，min，last？可以直接取表，而不用query*/

Loaded data with:
41214 Recordings
248 Engines
21 Sensor Measurements
3 Operational Settings
['engine_no', 'MAX(operational_setting_1)', 'MAX(rcm_operational_setting_1)']
['engine_no', 'MAX(recordings.operational_setting_1)', 'MAX(recordings.operational_setting_2)', 'MAX(recordings.operational_setting_3)', 'MAX(recordings.sensor_measurement_1)', 'MAX(recordings.sensor_measurement_10)', 'MAX(recordings.sensor_measurement_11)', 'MAX(recordings.sensor_measurement_12)', 'MAX(recordings.sensor_measurement_13)', 'MAX(recordings.sensor_measurement_14)', 'MAX(recordings.sensor_measurement_15)', 'MAX(recordings.sensor_measurement_16)', 'MAX(recordings.sensor_measurement_17)', 'MAX(recordings.sensor_measurement_18)', 'MAX(recordings.sensor_measurement_19)', 'MAX(recordings.sensor_measurement_2)', 'MAX(recordings.sensor_measurement_20)', 'MAX(recordings.sensor_measurement_21)', 'MAX(recordings.sensor_measurement_3)', 'MAX(recordings.sensor_measurement_4)', 'MAX(recordings.sensor_measurement_5)', 

In [10]:
# !cd demo && python3 predict.py # TODO
# no real-time data, just calc from fedb
# calc_sql = """
# select max(sensor_measurement_4) over w1, time_in_cycles 
# from t1 
# WINDOW w1 AS (PARTITION BY t1.time_in_cycles ORDER BY t1.record_time ROWS BETWEEN 100 PRECEDING AND CURRENT ROW);
# """
calc_sql = """(select * from t1 where engine_no = 1) as recordings last join cycles/*schema:time_in_cycles,MAX(...), including all engines*/
on recordings.time_in_cycles = cycles.time_in_cycles
;"""
# partition by不大行，目的是time_in_cycles一个值对应一堆max指标
# engine_no time_in_cycles sensors sensors_MAX_of_each_time_in_cycles

import sqlalchemy as db
engine = db.create_engine('fedb:///db_test?zk=127.0.0.1:2181&zkPath=/fedb')
connection = engine.connect()

from pandas import DataFrame, Series
import numpy as np
fm2 = DataFrame()

result = connection.execute(calc_sql)
print(result.rowcount)

for r in result:
#     print(r)
    fm2 = fm2.append([np.array(r).tolist()])

fm2.head()

230


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1.0,230.0,25.0070,0.6214,60.0,462.54,537.66,1264.31,1046.41,7.05,...,7890.31,10.7615,0.02,308.0,1915.0,84.93,14.41,8.6329,229.0,9.468222e+11
0,1.0,229.0,42.0055,0.8400,100.0,445.00,549.45,1356.32,1129.66,3.91,...,8109.04,9.2439,0.02,331.0,2212.0,100.00,10.54,6.3622,228.0,9.468216e+11
0,1.0,228.0,42.0010,0.8418,100.0,445.00,549.45,1352.89,1129.70,3.91,...,8112.49,9.2161,0.02,332.0,2212.0,100.00,10.65,6.5274,227.0,9.468210e+11
0,1.0,227.0,35.0068,0.8414,100.0,449.44,555.47,1369.11,1133.25,5.48,...,8092.23,9.1952,0.02,335.0,2223.0,100.00,14.90,8.9485,226.0,9.468204e+11
0,1.0,226.0,0.0018,0.0019,100.0,518.67,642.33,1592.22,1406.67,14.62,...,8162.71,8.2953,0.03,394.0,2388.0,100.00,39.46,23.5819,225.0,9.468198e+11


In [8]:
X = fm2.copy().fillna(0)
y = pd.read_csv(
    'demo/data/RUL_FD004.txt',
    sep=' ',
    header=None,
    names=['remaining_useful_life'],
    index_col=False,
)

# preds2 = reg.predict(X)
# mae = mean_absolute_error(preds2, y)
# print('Mean Abs Error: {:.2f}'.format(mae))

y.head()

,remaining_useful_life
0,22
1,39
2,107
3,75
4,149
